In [1]:
import csv
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, recall_score, precision_recall_curve, make_scorer
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import datetime
from random import randint
%pylab inline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

Populating the interactive namespace from numpy and matplotlib


/home/rashi/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['randint']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# get some data
x_train = np.loadtxt('files/x_train.dat')
y_train = np.loadtxt('files/y_train.dat')

model = RandomForestClassifier()
print(model.get_params().keys())

dict_keys(['bootstrap', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])


In [3]:
#Custom scoring function:
def my_recall(y_test, y_pred):
# Function goes in here
    return recall_score(y_test, y_pred, average="binary")

my_scorer = make_scorer(my_recall, greater_is_better=True,needs_proba=True)

In [4]:
param_grid = {'n_estimators': np.arange(10,501,40), 
              'min_samples_leaf':np.arange(2,10), 
              'warm_start':[True, False],
              'max_depth': np.arange(1,21,2),
              'max_features': np.arange(3,x_train.shape[1])}

rsearch = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100, scoring='recall')
rsearch.fit(x_train, y_train)
print(rsearch)
# summarize the results of the random parameter search

print("#### best recall:", rsearch.best_score_)
print("#### best n_estimators:", rsearch.best_estimator_.n_estimators)
print("#### best min_sample_leaf:", rsearch.best_estimator_.min_samples_leaf)
print("#### best warm_start:", rsearch.best_estimator_.warm_start)
print("#### best max_depth", rsearch.best_estimator_.max_depth)
print("#### best max_features:", rsearch.best_estimator_.max_features)


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'n_estimators': array([ 10,  50,  90, 130, 170, 210, 250, 290, 330, 370, 410, 450, 490]), 'min_samples_leaf': array([2, 3, 4, 5, 6, 7, 8, 9]), 'warm_start': [True, False], 'max_depth': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]), 'max_features': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='recall', ve